IMPORTS AND CONSTANTS:

In [11]:
!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
import os
import pickle
import time
import torch
from torch import nn
from google.colab import drive
drive.mount('/content/drive')

num_inst=100
num_jobs=500
num_ref_jobs=500
u=0.1
v=0.3
fractions = [0.2]

common_path = "/content/drive/My Drive/EPIA_2023"
instances_path = f"{common_path}/instances/{u}_{v}"
apriori_model_path = f"{common_path}/ml_apriori_models"
conditional_model_path = f"{common_path}/ml_conditional_models"

CLASSES AND FUNCTIONS:

In [10]:
class Instance:

    def __init__(self, a_dict, a_u, a_v):

        # read the parameters

        self.num_jobs = len(a_dict["durations"])
        self.durations = torch.clone(a_dict["durations"])
        self.due_dates = torch.clone(a_dict["due_dates"])
        self.deadlines = torch.clone(a_dict["deadlines"])
        self.weights = torch.clone(a_dict["weights"])
        self.solution = torch.clone(a_dict["solution"])
        self.optimal_value = a_dict["optimal_value"]
        self.heuristic_value = a_dict["heuristic_value"]
        self.u = a_u
        self.v = a_v

        # scheduling params
        self.jobs_queue = self.create_jobs_queue()
        self.queue_index = 0

        self.is_decided = torch.zeros((self.num_jobs,), dtype=torch.bool)
        self.is_scheduled = torch.zeros((self.num_jobs,), dtype=torch.bool)
        _, self.non_scheduled = [torch.tensor(t) for t in zip(*sorted(zip(self.deadlines, range(self.num_jobs))))]
        self.schedule = torch.zeros((self.num_jobs,), dtype=torch.int) - 1
        self.schedule_index = 0

        self.time_moment = 0
        self.margin = 0
        self.cost = 0

    def clear_schedule(self):

        self.queue_index = 0

        self.is_decided = torch.zeros((self.num_jobs,), dtype=torch.bool)
        self.is_scheduled = torch.zeros((self.num_jobs,), dtype=torch.bool)
        _, self.non_scheduled = [torch.tensor(t) for t in zip(*sorted(zip(self.deadlines, range(self.num_jobs))))]
        self.schedule = torch.zeros((self.num_jobs,), dtype=torch.int) - 1
        self.schedule_index = 0

        self.time_moment = 0
        self.margin = 0
        self.cost = 0

    def compute_apriori_features(self):

        dur_sum = torch.sum(self.durations)
        features = torch.empty((self.num_jobs, 8), dtype=torch.double)
        features[:, 0] = self.weights / 100
        features[:, 1] = self.durations / 100
        features[:, 2] = (self.due_dates - self.u * dur_sum) / (self.v * dur_sum - self.u * dur_sum)
        features[:, 3] = (self.deadlines - self.u * dur_sum) / (1.1 * dur_sum - self.u * dur_sum)
        features[:, 4] = (self.weights / self.durations - 0.01) / (100 - 0.01)
        features[:, 5] = (self.weights - self.durations) / 100
        features[:, 6] = self.due_dates / self.deadlines
        features[:, 7] = (self.deadlines - self.due_dates) / (1.1 * dur_sum - self.u * dur_sum)
        return features

    def compute_conditional_features(self, r_jobs=0):

        if r_jobs > 0:
            ref_jobs = torch.randperm(self.num_jobs)[:r_jobs]
        else:
            ref_jobs = torch.arange(self.num_jobs)

        target_features = self.compute_apriori_features()
        ref_features = target_features[ref_jobs]

        point_part = ref_features.repeat(self.num_jobs, 1)
        vector_part = (target_features.unsqueeze(1) - ref_features).reshape(self.num_jobs * len(ref_jobs),
                                                                            target_features.size(dim=1))
        return torch.cat((point_part, vector_part), dim=1)

    def compute_predictions(self, a_apriori_model, a_conditional_model, a_num_ref_jobs):

        a_apriori_model.eval()
        a_conditional_model.eval()

        with torch.no_grad():
            my_softmax = nn.Softmax(dim=1)
            apriori_features = self.compute_apriori_features()
            apriori_probs = my_softmax(a_apriori_model(self.compute_apriori_features()))[:, 1]

            ref_features = apriori_features[0:a_num_ref_jobs]
            ref_probs = apriori_probs[0:a_num_ref_jobs]
            probs = torch.zeros((self.num_jobs,), dtype=torch.float64)
            for j in range(self.num_jobs):
                conditional_features = torch.cat((ref_features, apriori_features[j] - ref_features), dim=1)
                early_probs = my_softmax(a_conditional_model(torch.cat(
                    (conditional_features, torch.ones((conditional_features.size(0), 1), dtype=torch.float64)),
                    dim=1)))[:, 1]
                tardy_probs = my_softmax(a_conditional_model(torch.cat(
                    (conditional_features, torch.zeros((conditional_features.size(0), 1), dtype=torch.float64)),
                    dim=1)))[:, 1]
                probs[j] = torch.mean(early_probs * ref_probs + tardy_probs * (1 - ref_probs))

        return torch.round(probs).to(torch.int), probs

    def compute_early_predictions(self):
        return torch.ones((self.num_jobs,), dtype=torch.int), torch.ones((self.num_jobs,), dtype=torch.float64)

    def compute_random_predictions(self):
        return torch.randint(high=2, size=(self.num_jobs,), dtype=torch.int), torch.full((self.num_jobs,), 0.5)

    def create_jobs_queue(self):

        keys = torch.cat((self.due_dates, self.deadlines))
        values = torch.cat((torch.linspace(0, self.num_jobs - 1, self.num_jobs, dtype=torch.int),
                            torch.linspace(0, self.num_jobs - 1, self.num_jobs, dtype=torch.int)))
        tuples = zip(*sorted(zip(keys, values)))
        _, ordered_jobs = [torch.tensor(t) for t in tuples]
        return ordered_jobs

    def edf_test(self, a_job):

        if self.margin - self.durations[a_job] >= 0:
            self.margin -= self.durations[a_job]
            return True
        else:
            jobs_indexes = self.non_scheduled[self.non_scheduled != a_job]
            durations = self.durations[jobs_indexes]
            deadlines = self.deadlines[jobs_indexes]
            my_cumsum = self.time_moment + self.durations[a_job] + torch.cumsum(durations, dim=0)
            self.margin = torch.amin(deadlines - my_cumsum)
            if self.margin >= 0:
                return True
            else:
                return False

    def improve_by_ilp(self, a_preds, a_probs, a_num_ilp_jobs):

        r_jobs, fix_jobs = reduction_subset(a_probs, a_num_ilp_jobs)
        weights, durations, due_dates, deadlines = self.reduce(r_jobs, a_preds[r_jobs])
        ilp_preds, obj_val = solve_by_ilp(len(weights), weights, durations, due_dates, deadlines)
        preds = torch.clone(a_preds)
        if obj_val > -1:
            preds[fix_jobs] = ilp_preds
        return preds

    def next_undecided_job(self):

        # schedule intermediate tardy jobs
        self.queue_index += 1
        num_jobs = 2 * self.num_jobs
        while self.queue_index < num_jobs:
            job = self.jobs_queue[self.queue_index]
            if self.is_decided[job]:
                if not self.is_scheduled[job]:
                    self.schedule_job(job)
                self.queue_index += 1
            else:
                break

    def recompute_schedule_cost(self):

        durations = self.durations[self.schedule]
        my_cumsum = torch.cumsum(durations, dim=0)

        # calculating the cost
        weights_permutation = self.weights[self.schedule]
        is_early = my_cumsum <= self.due_dates[self.schedule]
        weights = weights_permutation[is_early]
        total_cost = torch.sum(weights)

        # calculating the penalty
        is_violated = my_cumsum > self.deadlines[self.schedule]
        num_violations = len(is_violated[is_violated])
        penalty_value = 500 * num_violations
        flag = True
        if num_violations > 0:
            flag = False

        return total_cost - penalty_value, flag

    def reduce(self, a_jobs, a_labels):

        # make "a_jobs" standing the first
        jobs = torch.arange(self.num_jobs)
        T_mask = torch.zeros(self.num_jobs, dtype=torch.bool).index_fill(0, a_jobs, True)
        F_mask = torch.logical_not(T_mask)
        jobs = torch.cat((jobs[T_mask], jobs[F_mask]))

        # reduction
        due_dates = self.due_dates[jobs]
        deadlines = self.deadlines[jobs]
        durs = self.durations[T_mask]
        for k in range(len(a_jobs)):
            D = due_dates[k] if a_labels[k] == 1 else deadlines[k]
            due_dates = torch.where(due_dates <= D, torch.minimum(due_dates, D - durs[k]), due_dates - durs[k])
            deadlines = torch.where(deadlines <= D, torch.minimum(deadlines, D - durs[k]), deadlines - durs[k])

        weights = self.weights[F_mask]
        durations = self.durations[F_mask]
        due_dates = due_dates[len(a_jobs):]
        deadlines = deadlines[len(a_jobs):]
        return weights, durations, due_dates, deadlines

    def render(self):
        info = (
            f"RENDERING...\n"
            f"=== Given input data ===\n"
            f"Durations = {self.durations[self.non_scheduled]} \n"
            f"Due dates = {self.due_dates[self.non_scheduled]} \n"
            f"Deadlines = {self.deadlines[self.non_scheduled]} \n"
            f"Weights = {self.weights[self.non_scheduled]} \n"
            f"Solution = {self.solution[self.non_scheduled]} \n"
            f"Opt value = {self.optimal_value} \n"
            f"Heu value = {self.heuristic_value} \n"
            f"\n"
            f"=== Scheduling params ===\n"
            f"Jobs queue = {self.jobs_queue} \n"
            f"Queue index = {self.queue_index} \n"
            f"Schedule = {self.schedule} \n"
            f"Schedule index = {self.schedule_index} \n"
            f"Non scheduled jobs = {self.non_scheduled} \n"
            f"Time moment = {self.time_moment} \n"
            f"Margin = {self.margin} \n"
            f"Cost = {self.cost} \n"
            f"\n"
        )

        print(info)

    def step(self, a_job, a_prediction):

        is_early_job = (a_prediction == 1) and (self.edf_test(a_job))
        self.is_decided[a_job] = True
        if is_early_job:
            # a job becomes decided and being scheduled immediately
            self.is_scheduled[a_job] = True
            self.schedule_job(a_job)

        self.next_undecided_job()

    def schedule_job(self, a_job):

        self.schedule[self.schedule_index] = a_job
        self.schedule_index += 1
        self.non_scheduled = self.non_scheduled[self.non_scheduled != a_job]
        self.time_moment += self.durations[a_job]
        if self.time_moment <= self.due_dates[a_job]:
            self.cost += self.weights[a_job]

    def schedule_jobs(self, a_predictions):

        num_jobs = 2 * self.num_jobs
        while self.queue_index < num_jobs:
            current_job = self.jobs_queue[self.queue_index]
            pred = a_predictions[current_job]
            self.step(current_job, pred)

    def solve_by_nor(self):

        self.clear_schedule()
        _, indices = torch.sort(torch.cat((self.due_dates, self.deadlines)), stable=True)
        non_scheduled = torch.ones((self.num_jobs,), dtype=bool)

        cur_element = 0
        cur_scheduled = 0
        while cur_scheduled < self.num_jobs:
            job = indices[cur_element]
            if non_scheduled[job]:
                self.schedule[cur_scheduled] = job
                cur_scheduled += 1
            cur_element += 1

        self.recompute_schedule_cost()


class NeuralNetwork_Apriori(nn.Module):
    def __init__(self):
        super(NeuralNetwork_Apriori, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(8, 32),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.linear_stack(x)


class NeuralNetwork_Conditional(nn.Module):
    def __init__(self):
        super(NeuralNetwork_Conditional, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(17, 64),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        return self.linear_stack(x)


def collect_stats(a_instances, a_apriori_model, a_cond_model, a_fraction):
    print("Call for collecting stats. Fraction = " + str(a_fraction) + ":")
    our_gap, early_gap, rand_gap, heu_gap = float(0), float(0), float(0), float(0)
    our_opts, early_opts, rand_opts, heu_opts = float(0), float(0), float(0), float(0)
    inst_counter = 0
    for item in a_instances:
        print("Instance " + str(inst_counter + 1) + "...")
        inst = Instance(item, u, v)
        # compute predictions
        our_preds, our_probs = inst.compute_predictions(a_apriori_model, a_cond_model, num_ref_jobs)
        early_preds, early_probs = inst.compute_early_predictions()
        rand_preds, rand_probs = inst.compute_random_predictions()
        # call ILP
        our_preds = inst.improve_by_ilp(our_preds, our_probs, int(a_fraction * inst.num_jobs))
        early_preds = inst.improve_by_ilp(early_preds, early_probs, int(a_fraction * inst.num_jobs))
        random_preds = inst.improve_by_ilp(rand_preds, rand_probs, int(a_fraction * inst.num_jobs))
        # schedule jobs
        inst.schedule_jobs(our_preds)
        our_gap = our_gap + 1 - inst.cost / inst.optimal_value
        our_opts = our_opts + 1 if inst.cost >= inst.optimal_value else our_opts
        inst.clear_schedule()
        inst.schedule_jobs(early_preds)
        early_gap = early_gap + 1 - inst.cost / inst.optimal_value
        early_opts = early_opts + 1 if inst.cost >= inst.optimal_value else early_opts
        inst.clear_schedule()
        inst.schedule_jobs(rand_preds)
        rand_gap = rand_gap + 1 - inst.cost / inst.optimal_value
        rand_opts = rand_opts + 1 if inst.cost >= inst.optimal_value else rand_opts
        # the same for heuristic
        heu_gap = heu_gap + 1 - inst.heuristic_value / inst.optimal_value
        heu_opts = heu_opts + 1 if inst.heuristic_value >= inst.optimal_value else heu_opts
        inst_counter += 1
    our_gap = round(float((our_gap / inst_counter) * 100), 5)
    early_gap = round(float((early_gap / inst_counter) * 100), 5)
    rand_gap = round(float((rand_gap / inst_counter) * 100), 5)
    heu_gap = round(float((heu_gap / inst_counter) * 100), 5)
    our_opts = int((our_opts / inst_counter) * 100)
    early_opts = int((early_opts / inst_counter) * 100)
    rand_opts = int((rand_opts / inst_counter) * 100)
    heu_opts = int((heu_opts / inst_counter) * 100)
    return f"{num_jobs} & {our_opts} & {heu_opts} & {rand_opts} & {early_opts} & {our_gap} & {heu_gap} & {rand_gap} & {early_gap}"


def load(a_num_inst, a_num_jobs, a_U, a_V):
    input_path = os.path.join(f"{instances_path}/{a_num_jobs}.pkl")
    my_file = open(input_path, "rb")
    my_dict = pickle.load(my_file)
    my_file.close()
    instances = []
    counter = 0
    for item in my_dict.values():
        if counter < a_num_inst and len(item["solution"]) > 0 and item["heuristic_value"] > -1:
            instances.append(item)
            counter += 1
    print(str(counter) + " instances found")
    return instances


def reduction_subset(a_probs, a_num_ilp_jobs):
    _, indices = torch.sort(torch.abs(a_probs - 0.5))
    return torch.unique(indices[a_num_ilp_jobs:]), torch.unique(indices[:a_num_ilp_jobs])


def solve_by_ilp(a_num_jobs, a_weights, a_durations, a_duedates, a_deadlines):
    if a_num_jobs == 0:
        return torch.tensor([]), -1
    jobs = torch.arange(a_num_jobs)
    time_moments = torch.unique(torch.cat((a_duedates, a_deadlines)))
    m = gp.Model(name="Maximizing Weighted Number Of Jobs")
    m.Params.OutputFlag = 0
    m.Params.FeasibilityTol = 1e-09
    m.Params.OptimalityTol = 1e-09
    m.Params.timelimit = 60.0
    x = m.addVars(a_num_jobs, vtype=GRB.BINARY, name="x")
    m.setObjective(gp.quicksum(a_weights[j] * x[j] for j in range(a_num_jobs)), GRB.MAXIMIZE)
    for t in time_moments:
        m.addConstr(gp.quicksum(a_durations[i] for i in jobs[a_deadlines <= t])
                    + gp.quicksum(
            a_durations[i] * x[int(i)] for i in jobs[torch.logical_and(a_deadlines > t, a_duedates <= t)]) <= t)
    m.optimize()

    if m.Status == GRB.OPTIMAL:
        my_vars = torch.empty((a_num_jobs,), dtype=torch.int)
        for i in range(a_num_jobs):
            my_vars[i] = x[i].x
        return my_vars, int(m.objVal)
    else:
        return torch.tensor([]), -1

PERFORM COMPUTATION:

In [11]:
instances = load(num_inst, num_jobs, u, v)

apriori_model = NeuralNetwork_Apriori().double()
apriori_model.load_state_dict(torch.load(f"{apriori_model_path}/{u}_{v}.pt"))

cond_model = NeuralNetwork_Conditional().double()
cond_model.load_state_dict(torch.load(f"{conditional_model_path}/{u}_{v}.pt"))

my_results = []
for fract in fractions:
    my_results.append(collect_stats(instances, apriori_model, cond_model, fract))
for item in my_results:
    print(item)